In [1]:
import os
import numpy as np
import glob
import random

from preprocessing import *
from visualize import *

from matplotlib import pyplot as plt

In [2]:
model_name = 'dcc_crowdnet'
model_path = os.path.expanduser(os.path.join('models', model_name))
data_path = os.path.expanduser(os.path.join('~/data/', model_name))
weights_path = os.path.expanduser(os.path.join('~/models/', model_name))

dataset_paths = ['dataset/UCF_CC_50/']

slice_w = 256
slice_h = 256

patch_w = 225
patch_h = 225

net_density_h = 28
net_density_w = 28

In [3]:
#mean
VGG_ILSVRC_16_layers_mean = np.zeros((3, patch_h, patch_w), dtype='f4')
VGG_ILSVRC_16_layers_mean[0,:,:] = 103.939
VGG_ILSVRC_16_layers_mean[1,:,:] = 116.779
VGG_ILSVRC_16_layers_mean[2,:,:] = 123.68

In [4]:
# Positive image and ground truth loading
X_fs = []
Y_fs = []

for path in dataset_paths:
    images, gts, densities = load_images_and_gts(path)
    X_fs += images
    Y_fs += densities

dataset/UCF_CC_50/20.json
dataset/UCF_CC_50/20.json 62.50520805512529 637.2357512953367 (<class 'IndexError'>, IndexError('index 637 is out of bounds for axis 1 with size 600',), <traceback object at 0x117b84488>)
(384, 600)
dataset/UCF_CC_50/36.json
(768, 1024)
dataset/UCF_CC_50/41.json
(656, 1024)
dataset/UCF_CC_50/16.json
(600, 800)
dataset/UCF_CC_50/6.json
(768, 1024)
dataset/UCF_CC_50/7.json
(424, 600)
dataset/UCF_CC_50/17.json
dataset/UCF_CC_50/17.json 395.77461139896366 1024.3341968911911 (<class 'IndexError'>, IndexError('index 1024 is out of bounds for axis 1 with size 1024',), <traceback object at 0x1178eba88>)
(768, 1024)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-8853e0f5002d>", line 6, in <module>
    images, gts, densities = load_images_and_gts(path)
  File "/Users/samuelcahyawijaya/htdocs/python/crowd_counter/preprocessing.py", line 59, in load_images_and_gts
    density = gaussian_filter_density([gt])[0]
  File "/Users/samuelcahyawijaya/htdocs/python/crowd_counter/preprocessing.py", line 237, in gaussian_filter_density
    density += scipy.ndimage.filters.gaussian_filter(pt2d, sigma, mode='constant')
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/ndimage/filters.py", line 344, in gaussian_filter
    mode, cval, truncate)
  File "/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3

KeyboardInterrupt: 

In [ ]:
# Split test an train set
from sklearn.model_selection import train_test_split

# Train 80%, Test 20%
X_fs_train, X_fs_test, Y_fs_train, Y_fs_test = train_test_split(X_fs, Y_fs, test_size=0.2)

In [ ]:
# FS: FULL SIZE
X_train, Y_train = X_fs_train, Y_fs_train
X_test, Y_test = X_fs_test, Y_fs_test

# DATA AUGMENTATION
print ('\nMultiscale pyramidal')
print ('TRAIN:')
X_train, Y_train = multiscale_pyramidal(X_train, Y_train)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = multiscale_pyramidal(X_test, Y_test)
print (len(X_test), len(Y_test))

# PATCH SIZE
print ('\nGenerate slices')
print ('TRAIN:')
X_train, Y_train = generate_slices(X_train, Y_train, slice_w=patch_w, slice_h=patch_h, offset=8)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = generate_slices(X_test, Y_test, slice_w=patch_w, slice_h=patch_h)
print (len(X_test), len(Y_test))

print ('\nFlip')
print ('TRAIN:')
X_train, Y_train = flip_slices(X_train, Y_train)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = flip_slices(X_test, Y_test)
print (len(X_test), len(Y_test))

print ('\nSamples gt distribution correction')
print ('TRAIN:')
X_train, Y_train = samples_distribution(X_train, Y_train)
print (len(X_train), len(Y_train))

print ('\nShuffle')
print ('TRAIN:')
X_train, Y_train = shuffle_slices(X_train, Y_train)
print (len(X_train), len(Y_train))
print ('TEST:')
X_test, Y_test = shuffle_slices(X_test, Y_test)
print (len(X_test), len(Y_test))

In [ ]:
vis_square(list_to_np_array(X_train[:20]))

In [ ]:
Y_train_sum = list(filter(lambda x: True, map(np.sum, Y_train)))
Y_train_sum.sort()
plt.plot(Y_train_sum)
plt.show()

In [ ]:
# Train
print ('TRAIN:')
process_dump_to_hdf5data(X_train, Y_train, data_path, 'train', VGG_ILSVRC_16_layers_mean)
# Test
print ('TEST:')
process_dump_to_hdf5data(X_test, Y_test, data_path, 'test', VGG_ILSVRC_16_layers_mean)